# Technical Test 01

## Database Schema Design

>Q1.1. How can we link "Task" to an element of type either "ItemA" either "ItemB"? An explanation or a schema (drawing) will suffice.

We can place the PK for ItemA and PK for ItemB in the Items table with either A or B as optional. However, it would violate 1NF of database normalization as there would be repeating groups of data.

This would be more of a quick-and-dirty solution that would lead to technical debt in the future should more changes and complexity be needed.

A better solution would be the answer below for Q1.2

<img src="sql_schema_1.png" />

>Q1.2. How to modify the Task table so that each task element can be link to an item of any type (ItemA, ItemB, .. , ItemM, ItemN, ...).

We can use a one-to-many non-optional relationship to isolate the ItemType of each Item, allowing an Item to be either A, B, C... but not allowing an Item to be both A or B at the same time.

This would ensure that data is:
- without repeating groups (1NF)
- data is dependant on key (2NF)
- without redundant data (3NF)

<img src="sql_schema_2.png" />

# Cleaning Strings
>Q2.1. Write a function spaceCorrect that will take in input the text with errors (string), and will return as output the corrected text (string).

In [1]:
dirty_input = 'M.   Charles Granger , hello?Are you there?!?'

In [2]:
import re
def _transform(group):
    """Accepts a string character group, iterates over each character in the group and adds spacing for punctuation rules.
    
    Args:
        group (string): the string to transform.
        
    Returns:
        string: transformed string with correct punctuation rules
    """
    result = re.sub(r' *([.,]) *', lambda x: '{} '.format(x.group().strip()), group)
    result = re.sub(r' *([?!:;]) *', lambda x: ' {} '.format(x.group().strip()), result)
    return result

def _fix_double_space(group):
    """Accepts a string character group, finds a double space in it and replaces it with a single space.
    
    Args:
        group (string): the string to transform.
    
    Returns:
        string: string without double spaces.
    """
    if group.find('  ') > -1:
        fixed = group.replace('  ', ' ')
    return fixed

def spaceCorrect(text):
    """Corrects spacing in a string according to punctuation rules
    Args:
        text (string): the string to correct
    
    Returns:
        string: the corrected string
    """
    # splits the input string by spaces to separate character groups
#     groups = text.split(' ')
    
    #filters out empty strings
#     no_empty_string = list(filter(lambda x: x != '', groups))
    transformed = _transform(text)
    # iterates over each character group and transforms them to correct punctuation
#     with_space_after = [_transform(x) for x in no_empty_string]
    
    #joins back each character group with spaces between them
#     joined = ' '.join(with_space_after)
    
    # before returning the result, we clean the string once more to fix double spacing
    # which may be due to 2 symbols being next to each other
    return _fix_double_space(transformed)

print(spaceCorrect(dirty_input))

M. Charles Granger, hello ? Are you there ? ! ? 


>Q2.2. Write a function betterSpaceCorrect(text, correspondances) that will do the same job as in Q2.1, using this associative array as input.

In [3]:
correspondances = {
    ",": [0, 1],
    ".": [0, 1],
    ":": [1, 1],
    ";": [1, 1],
    "!": [1, 1],
    "?": [1, 1]
}

In [4]:
def _better_transform(group, correspondances):
    """Accepts a string character group, iterates over each character in the group and adds spacing for punctuation rules.
    
    References a correspondances dictionary for punctuation rules
    
    Args:
        group (string): the string to transform.
        correspondances (dict): dictionary that specifies the punctuation rules.
            Format of punctuation corresponds to [before, after]
        
    Returns:
        string: transformed string with correct punctuation rules
    """
#     result = []
#     for char in group:
#         if char in correspondances:
#             #symbol with punctuation rule found
#             result.append('{before}{character}{after}'.format(
#                 before = ' '*correspondances[char][0],
#                 character = char,
#                 after = ' '*correspondances[char][1]))
#         else:
#             # no symbol with punctuation rule found
#             result.append(char)
#     return ''.join(result)
    result = re.sub(r' *([.,?!:;]) *', lambda x: '{before}{character}{after}'.format(
        before = ' '*correspondances[x.group().strip()][0],
        character = x.group().strip(),
        after = ' '*correspondances[x.group().strip()][1]), group)
    
    #             result.append('{before}{character}{after}'.format(
#                 before = ' '*correspondances[char][0],
#                 character = char,
#                 after = ' '*correspondances[char][1]))
#     result = re.sub(r' *([?!:;]) *', lambda x: ' {} '.format(x.group().strip()), result)
    return result

In [5]:
def betterSpaceCorrect(text, correspondances):
    """Corrects spacing in a string according to punctuation rules, with corespondances used instead of hard-coded rules.
    
    Args:
        text (string): the string to correct
        correspondances (dict): dictionary that specifies the punctuation rules.
            Format of punctuation corresponds to [before, after]
        
    Returns:
        string: the corrected string
    """
    # splits the input string by spaces to separate character groups
#     groups = text.split(' ')
    
    #filters out empty strings
#     no_empty_string = list(filter(lambda x: x != '', groups))
    
    # iterates over each character group and transforms them to correct punctuation
#     with_space_after = [_better_transform(x, correspondances) for x in no_empty_string]
    
    #joins back each character group with spaces between them
#     joined = ' '.join(with_space_after)
    transformed = _better_transform(text, correspondances)
    
    # before returning the result, we clean the string once more to fix double spacing
    # which may be due to 2 symbols being next to each other
    return _fix_double_space(transformed)

print(betterSpaceCorrect(dirty_input, correspondances))

M. Charles Granger, hello ? Are you there ? ! ? 


>Q2.3. Enhance spaceCorrect to take lists into account.

In [6]:
dirty_input_with_lists = """M.   Charles Granger , hello?Are you there?!?

- premier point ;
- deuxième point   .

- deuxième point,

- dernier point   ;"""

In [7]:
def _fix_lists(text):
    """substitutes string sthat match the regex pattern and invokes the lambda function to format each match object"""
    pattern = r'\n*(- )(.+)([.,;])'
    formatted = re.sub(pattern, lambda x: _reformat_list(x), text )
    return formatted

def _reformat_list(list_item):
    """Formats the list item regex match object provided"""
    ending_symbol = list_item.group(3)
    
    #replaces the ending symbol if it is not a full stop or a semi-colon
    if not ending_symbol in ['.',';']:
        ending_symbol = ';'
    
    #formats the list item nicely
    result = '\n{dash}{item}{end}'.format(
        dash = list_item.group(1),
        item = list_item.group(2).rstrip(),
        end = ending_symbol)
    return result

In [8]:
def enhancedSpaceCorrect(text, correspondances):
    """Corrects spacing in a string according to punctuation rules, also taking into account lists.
    
    Args:
        text (string): the string to correct
        correspondances (dict): dictionary that specifies the punctuation rules.
            Format of punctuation corresponds to [before, after]
        
    Returns:
        string: the corrected string
    """
    # splits the input string by spaces to separate character groups
#     groups = text.split(' ')
    
    #filters out empty strings
#     no_empty_string = list(filter(lambda x: x != '', groups))
    
    # iterates over each character group and transforms them to correct punctuation
#     with_space_after = [_transform(x) for x in no_empty_string]
    
    #joins back each character group with spaces between them
    joined = _transform(text)
    
    #fix lists
    fixed_lists = _fix_lists(joined)
    
    # before returning the result, we clean the string once more to fix double spacing
    # which may be due to 2 symbols being next to each other
    return _fix_double_space(fixed_lists)
print(enhancedSpaceCorrect(dirty_input_with_lists, correspondances))

M. Charles Granger, hello ? Are you there ? ! ? 
- premier point; 
- deuxième point. 
- deuxième point; 
- dernier point; 


# Front End, HTML Form operations
>Write a Javascript (or Coffeescript) plugin which suggests to the user to choose between all "fondateur" persons with a click and then fill automatically the "president" fieldset using the "data-from" attribute.

>Any external plugin is welcome (jQuery, Zepto, Underscore, etc.)

>Here's a fiddle to help and start the work: http://jsfiddle.net/lajarre/VD3mY/4/

Answer in React is here: https://jsfiddle.net/ziinc/yhz3grfd/5/